In [60]:
import pandas as pd
import pickle
import os
from glob import glob
import soynlp
from soyspacing.countbase import CountSpace
from konlpy.tag import Mecab
from gensim.models import FastText
mecab = Mecab()

In [70]:
os.chdir('/data/kmkim/chatbot_data_model/dental_5/5_data') ### 생성된 데이터들이 있는 주소로 변경 필요
data_list= sorted(glob('*.csv'))
print(data_list)

['patient_1_patient_data.csv', 'patient_2_patient_data.csv', 'patient_3_patient_data.csv', 'patient_4_patient_data.csv', 'patient_5_patient_data.csv']


In [51]:
def soytext_Fastembedding(loaded_data, file_path, name):
    
    with open(file_path + name + '.pkl','wb') as f:
        pickle.dump(loaded_data, f)
    
    ## 맞춤법 학습용 soy model
    with open(file_path + name + '_soy.txt', 'w', encoding = 'utf-8') as f:
        for i in loaded_data:
            try:
                f.write(i[0] +'\n')
                f.write(i[1] +'\n')
            except TypeError:
                print(i)
    corpus_fname = file_path + name +'_soy.txt'
    model = CountSpace()
    model.train(corpus_fname)
    model_fname = file_path + name  +'_soy.model'
    model.save_model(model_fname, json_format= False) ## 맞춤법 모델 학습
    
    
    ##FastText 임베딩 과정
    Fast_list = []
    for i in loaded_data :
        tokens1 = mecab.morphs(i[0])
        tokens2 = mecab.morphs(i[1])
        Fast_list.append(tokens1)
        Fast_list.append(tokens2)
    model = FastText(sentences = Fast_list, vector_size = 256, window= 2, min_count= 1, workers= 4, 
                 sg = 1, epochs = 20, hs= 1,)
    
    print()
    print('---- FastText Word Embedding is over')


    model.wv.save_word2vec_format(file_path + 'FastText_' + name)

In [50]:
def dataformat_preprocessing(data_list):
    for file in data_list:
        file_name = file
        x_df = pd.read_csv(file_name, header = None)

        pair_list = [[] for n in range(len(x_df))]

        for i in range(len(x_df)):
            pair_list[i].append(x_df[0][i])
            pair_list[i].append(x_df[1][i])

        soytext_Fastembedding(pair_list, 'prepro/', file_name[:-4]) ## 'prepro/' 은 코드 실행을 통해 생성된 데이터들이 저장되는 주소임. 향후 각 환경에 맞게 수정 가능 

In [72]:
dataformat_preprocessing(data_list) ## 각 환자 데이터에 대한 전처리 및 임베딩 생성

all tags length = 31357 --> 24717, (num_doc = 37907)
---- FastText Word Embedding is over
